In [3]:
print("문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드")

문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드


In [13]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

gs
3g


In [ ]:
#pip install langchain-community

  Using cached langchain_community-0.3.29-py3-none-any.whl.metadata (2.9 kB)
  Using cached aiohttp-3.12.15-cp313-cp313-win_amd64.whl.metadata (7.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached frozenlist-1.7.0-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached multidict-6.6.4-cp313-cp313-win_amd64.whl.metadata (5.4 kB)
  Using cached propcache-0.3.2-cp313-cp313-win_amd64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp313-cp313-win_amd64.whl.metadata (76 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata

In [8]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 로드
loader = PyPDFLoader('./data/콘텐츠분쟁해결_사례.pdf')
print(loader)
docs = loader.load()

print(f"로드된 문서 수: {len(docs)}")
print(type(docs[0]))
print(docs[0].page_content[:300])  # 첫 300자 미리보기


로드된 문서 수: 109
<class 'langchain_core.documents.base.Document'>



In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,      # 권장 범위
    chunk_overlap=300,    # 맥락 보존
    separators=[
        "\n【사건개요】",
        "\n【쟁점사항】",
        "\n【처리경위】",
        "\n【처리결과】",
        "\n", "\n\n", "\n", ".", " ", ""
    ]
)

# 문서 분할
splits = text_splitter.split_documents(docs)

print(f"총 분할된 청크 수: {len(splits)}")
print("첫 번째 청크 미리보기:\n")
print(splits[0].page_content[:300])


총 분할된 청크 수: 104
첫 번째 청크 미리보기:

콘텐츠분쟁조정 법리 연구 2부
- 타 분쟁조정사례 조사 -


In [14]:
from langchain_upstage import UpstageEmbeddings

# 임베딩 모델 초기화
embedding = UpstageEmbeddings(model="solar-embedding-1-large", api_key=UPSTAGE_API_KEY)

print("임베딩 모델 준비 완료:", embedding)


임베딩 모델 준비 완료: client=<openai.resources.embeddings.Embeddings object at 0x0000021421180050> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000021421180980> model='solar-embedding-1-large' dimensions=None upstage_api_key=SecretStr('**********') upstage_api_base='https://api.upstage.ai/v1/solar' embedding_ctx_length=4096 embed_batch_size=10 allowed_special=set() disallowed_special='all' chunk_size=1000 max_retries=2 request_timeout=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers={'x-upstage-client': 'langchain'} default_query=None http_client=None http_async_client=None


In [16]:
from langchain_community.vectorstores import FAISS

# 분할된 문서를 벡터화하여 저장
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)

print("벡터 저장소 생성 완료")


벡터 저장소 생성 완료


In [17]:
# 검색기 설정
retriever = vectorstore.as_retriever(
    search_type="similarity",   # 또는 "mmr" (다양성 고려)
    search_kwargs={"k": 5}      # 상위 5개 문서 검색
)

print("검색기 준비 완료!")


검색기 준비 완료!


In [19]:
from langchain_upstage import ChatUpstage

# LLM 초기화
llm = ChatUpstage(
    model="solar-pro",
    base_url="https://api.upstage.ai/v1",
    temperature=0.2
)

print("LLM 준비 완료")


LLM 준비 완료


In [21]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다.
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

print("QA 체인 준비 완료")


QA 체인 준비 완료


In [22]:
query = "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?"

result = qa_chain.invoke({"query": query})

print("\n[질문]")
print(query)
print("\n[답변]")
print(result["result"])
print("\n[참조 문서]")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"[{i}] {doc.metadata} → {doc.page_content[:200]}...")



[질문]
온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?

[답변]
### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 복구 거부 시 해결 방안  

#### 1. **사례 분석을 통한 쟁점 정리**  
제시된 사례집에 따르면, 시스템 오류로 인한 아이템 복구 문제는 다음과 같은 쟁점으로 구분됩니다:  
- **(1) 시스템 오류의 객관적 입증 여부** (2009년 사례 참조)  
  - 다수의 사용자에게 동일한 오류가 발생했는지, 게임 로그 등으로 오류 사실을 입증할 수 있어야 합니다.  
  - 단일 사용자의 주장만으로는 회사가 오류를 인정하지 않을 가능성이 높습니다.  
- **(2) 계정 소유권 및 약관 위반 여부** (2006년 사례 참조)  
  - 게임사가 현금거래 또는 계정 공유를 금지한 경우, 계정 명의자가 아닌 실소유자의 복구 요청은 거절될 수 있습니다.  
- **(3) 게임사의 복구 의무 및 약관 조항** (2006년 프로그램 오류 사례 참조)  
  - 게임사가 프로그램 오류를 인정하고 복구한 사례가 있으나, 이는 회사의 재량에 따른 것입니다.  

#### 2. **관련 법령 및 약관 검토**  
- **민법 제250조(도품·유실물 특례)**  
  - 아이템이 해킹 또는 시스템 오류로 소멸된 경우, 게임사가 "유실물 반환의무"를 질 수 있으나, **게임머니는 금전으로 간주되어 적용되지 않을 수 있습니다** (2009년 사례 참조).  
- **전자상거래법 및 약관 규제법**  
  - 게임사의 약관이 불공정할 경우(예: "모든 책임은 사용자에게 있다"는 조항), 무효 또는 수정 요구가 가능합니다.  
  - **약관에 시스템 오류 시 복구 절차가 명시되어 있는지 확인**해야 합니다.  

#### 3. **실무적 해결 단계**  
**(1) 증거 수집 및 오류 입증**  
- 게임 내 로그, 스크린샷, 고객센터 대화 기록 등을 보관하여 시스템 오류를 입